# Python MapReduce Example

Word count implemented in pure Python.

This notebook exemplifies the execution of a map-reduce program in Python, using Hadoop.
In this example, hadoop runs in standalone mode and reads data from the local filesystem, while in cluster mode data is read typically from HDFS dsitributed file system.


### Download the dataset 

In [ ]:
!wget -q -O os_maias.txt https://www.dropbox.com/s/n24v0z7y79np319/os_maias.txt?dl=0

## WordCount Example
Read the words from input and count them.

The processing is split into two steps:

+ The mapper emits for each line the number of words
+ The reduces sums all the tuples produced by the mapper stage...

### Mapper

By starting an element with "%%file", you are specifying that when run, the contents are written to the local disk.

In [ ]:
%%file mapper.py
#!/usr/bin/env python

# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # remove punctuation characters
    line = line.translate(str.maketrans('', '', string.punctuation+'«»'))
    # split the line into words
    words = line.split()
    print('words\t%s' % len(words))

Writing mapper.py


### Reducer

In [ ]:
%%file reducer.py
#!/usr/bin/env python

import sys

total_count = 0

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    key, count = line.split('\t', 1)

    # convert count (currently a string) to int
    count = int(count)

    total_count += count

print('words\t%s' % (total_count))

Writing reducer.py


## Local execution

The scripts can be tested using just the unix shell, as follows...

### Make the scripts executable

In [ ]:
!chmod a+x mapper.py && chmod a+x reducer.py

### Execute

The execution workflow is as follows:

+ The input file is piped into the input of the mapper;
+ The output the mapper is sorted;
+ The sorted output of the mapper is fed to the reducer stage.

In [ ]:
!cat "os_maias.txt" | ./mapper.py | sort -k1,1 | ./reducer.py

words	213359


## MapReduce with HADOOP

## Install Hadoop on Google Colab


In [ ]:
%%bash
HADOOP=hadoop-3.3.4
wget -q https://downloads.apache.org/hadoop/common/$HADOOP/$HADOOP.tar.gz
tar -xzf $HADOOP.tar.gz
mv $HADOOP/ /usr/local/
echo "export JAVA_HOME="`readlink -f /usr/bin/java | sed "s:bin/java::"` >> /usr/local/$HADOOP/etc/hadoop/hadoop-env.sh

echo "#!/bin/bash" > /usr/local/bin/hadoop
echo "exec /usr/local/$HADOOP/bin/hadoop \$*" >> /usr/local/bin/hadoop
chmod a+x /usr/local/bin/hadoop

## Hadoop standalone mode execution

For executing in an hadoop cluster, input data should be moved into an HDFS directory. For executing in standalone mode, data can be read from the local filesystem. 


The output directory needs to be cleared...

In [ ]:
rm -rf results

### Submitting the job

The _hadoop_ command is used to submit the mapreduce job to the cluster...

In [ ]:
!hadoop jar /usr/local/hadoop-3.3.4/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py -input os_maias.txt -output results

2022-09-19 23:33:17,222 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-09-19 23:33:17,477 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-09-19 23:33:17,477 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-09-19 23:33:17,506 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-09-19 23:33:17,836 INFO mapred.FileInputFormat: Total input files to process : 1
2022-09-19 23:33:17,861 INFO mapreduce.JobSubmitter: number of splits:1
2022-09-19 23:33:18,327 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local698779115_0001
2022-09-19 23:33:18,327 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-09-19 23:33:18,811 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local698779115_0001_12e661d6-6c4b-4ae9-9d21-d3cbd888cb0d/mapper.py
2022-09-19 23:33:18,846 INFO mapred.LocalDistributedCacheMana

#### Checking the results
The result is stored in directory results.

In [ ]:
!cat results/part-*

words	213359
